<a href="https://colab.research.google.com/github/KlaidasKaralevicius/NLP_lab_3/blob/main/lab3_Klaidas_Karalevicius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Notice** (prie paleidžiant notebook)
Apmokant modelį mažiau nei 5 minutes sumarizacija pateikia duoto teksto pradžią (žodis į žodį) tik sustoja anksčiau (apie 40 žodžių). Kad gauti patenkinamą rezultatą modelį reikėjo apmokyti and daugiau epochų (8 epochų davė patenkinamą rezultatą) todėl, kad atkartoti rezultatą reikia įkelti ilgiau apmokytą modelį. Instrukcija:


*   Atkomentuoti *!gdown* komandos bloką
*   Užkomentuoti *fit* komandos bloką
*   Reikiamus blokus galima rasti -> CTRL+F ir įvesti 'pointer'

**P.S.** Įkeliant jau apmokytą modelį užtenka paleisti tik 3 paskutinius notebook blokus (atkomentavus *!gdown*), nereikia nieko komentuoti, įkeldinėti ar tokenizuoti pradinių duomenų (užtenka naudoti CPU).

# Summarisation by finetuning

Sumarizacijos užduočiai reikia transformerio kuris turi ir enkoderį ir dekoderį, nes reikia sukurti naują tekstą sumarizacijai. Dėl to pasirinkau modelį kuris turi šias dvi dalis - BART (base model). Iš reikiamų modelių jis dažniausiai buvo paminėtas skirtinguose šaltiniuose. Iš pradžių buvo panaudotas didesnis BART modelis (bart-large-cnn), bet į jį galima paduoti tik labai mažus duomenų kiekis, kitu atveju visa RAM atmintis užsipildo ir notebook lūžta.

Duomenims fine-tunning užduočiai pasirinkau cnn_dailymail, iš pradžių pasirinkau XSum, bet jis buvo didesnis todėl ilgiau siuntėsi, apmokymui bus naudojama tik maža duomenų dalis, todėl neverta siūsti daugiau duomenų jei ne visi bus naudojami (duomenų paruošimas užims mažiau laiko).

Modelio tikslumui tikrinti panaudotas rouge_score, jis palygina gautą tekstą su testavimo tekstu ir tikrina kiek panašių n-gramų jie turi.

Import dependencies

In [ ]:
!pip install -U -q datasets transformers rouge_score evaluate nltk
from datasets import load_dataset
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, get_scheduler
import evaluate
from accelerate import Accelerator
import numpy as np
from tqdm.auto import tqdm
import random
random.seed(22)

Įkelti duomenis ir tokenizuoti duomenis (naudoti tik duomenų mažą dalį)

In [ ]:
dataset = load_dataset("cnn_dailymail", "3.0.0", trust_remote_code=True)

model_name = 'facebook/bart-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
n_samples = 2000
# iš visų duomenų pasirinkti tik atsitiktinius 2000, kad pagreitinti apmokymą
small_dataset_train = dataset['train'].shuffle().select(range(n_samples))
small_dataset_eval = dataset['test'].shuffle().select(range(n_samples))

In [ ]:
max_input_length = 512
max_target_length = 32

def preprocess_function(data):
  model_inputs = tokenizer(
      data['article'],
      max_length = max_input_length,
      truncation = True,
  )
  labels = tokenizer(
      data['highlights'],
      max_length = max_target_length,
      truncation = True
  )
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [ ]:
#tokenizuojami duomenys
tokenized_train_dt = small_dataset_train.map(preprocess_function, batched=True)
tokenized_eval_dt = small_dataset_eval.map(preprocess_function, batched=True)

Paruošti modelį ir duomenis

In [ ]:
rouge_score = evaluate.load('rouge')

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# iš tokenizuotų duomenų išmetami pradiniai stulpelių pavadinimai
tokenized_train_dt = tokenized_train_dt.remove_columns(
    small_dataset_train.column_names
)
tokenized_eval_dt = tokenized_eval_dt.remove_columns(
    small_dataset_eval.column_names
)

In [ ]:
tokenized_train_dt.set_format("torch")
tokenized_eval_dt.set_format("torch")

In [ ]:
# paruošiami duomenys įkelimui į modelio apmokymą ir testavimą
batch_size = 24
train_dataloader = DataLoader(
    tokenized_train_dt,
    shuffle = True,
    collate_fn = data_collator,
    batch_size = batch_size,
)
eval_dataloader = DataLoader(
    tokenized_eval_dt,
    collate_fn = data_collator,
    batch_size = batch_size,
)

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-5)

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
# paruošiami parametrai modelio apmokymui
num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

Modelio apmokymas ir testavimas

In [ ]:
# paruošiama funkciją modelio testavimui
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
    return preds, labels

In [ ]:
output_dir = "/content/bart-base-finetuned"

In [ ]:
# pointer
# apmokomas modelis
progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch) # įkeliami duomenys dalimis
        loss = outputs.loss # apskaičiuojamas loss
        accelerator.backward(loss) # apskaičiuojamas gradientas
        optimizer.step() # modelio parametrai keičiami pagal optimizatorių
        lr_scheduler.step() # keičiamas learning rate
        optimizer.zero_grad() # užnulina gradientą naujai iteracijai
        progress_bar.update(1)
# testuojamas modelis
model.eval()
for step, batch in enumerate(eval_dataloader):
    # apskaičiuojamas rouge tik pirmiem 2 porom, kad pagreitinti testavimą
    if step >= 1:
        break
    # atjungiamas gradient skaičiavimas
    with torch.no_grad():
        # generuojami spėjimai
        generated_tokens = accelerator.unwrap_model(model).generate(
            batch["input_ids"], attention_mask=batch["attention_mask"],
        )
        # padding
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens,
            dim = 1,
            pad_index = tokenizer.pad_token_id
        )

        labels = batch["labels"]
        # padding
        labels = accelerator.pad_across_processes(
            batch["labels"],
            dim = 1,
            pad_index = tokenizer.pad_token_id
        )

        generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
        labels = accelerator.gather(labels).cpu().numpy()
        # pakeisti -100 reikšmes, nes jų negalima dekoduoti
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]
        # išversti tokenus į tekstą
        decoded_preds = tokenizer.batch_decode(
            generated_tokens, skip_special_tokens=True
        )
        # išversti tokenus į tekstą
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        # panaudojama ankstesnė funkcija
        decoded_preds, decoded_labels = postprocess_text(
            decoded_preds, decoded_labels
        )
        # apskaičuojamas rouge
        rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)
        result = rouge_score.compute()
        result = {key: value for key, value in result.items()}
        result = {key: round(value, 4) for key, value in result.items()}
        print(f"{model_name}:", result)
# išsaugomas modelis
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
if accelerator.is_main_process:
    tokenizer.save_pretrained(output_dir)

Fine-tunned model testavimas ant naujų duomenų

In [ ]:
# pointer

# !gdown --fuzzy 'https://drive.google.com/file/d/1ftSBpUPsVtlswOzfc2HckbwMyx7GbrcY/view?usp=drive_link'
# !unzip bart-base-finetuned.zip

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization", model='bart-base-finetuned')

def print_summary(text):
    summary = summarizer(text)[0]['summary_text']
    print(f"> Text: {text}\n")
    print(f"> Summary: {summary}")

In [ ]:
text = '''
Physicists in Italy and China have for the first time observed glimmers of the ‘neutrino fog’, signals from neutrinos that mimic those expected to be produced by dark matter.
The observations are a double-edged sword, says Nicole Bell, a theoretical physicist at the University of Melbourne, Australia. On the one hand, it means that detectors have become sensitive enough to pick up signals of dark matter — the mysterious substance thought to make up the bulk of matter in the Universe. On the other, it means that the neutrino signals could obscure the dark-matter signals that scientists are so eager to observe. The findings were published in two papers in Physical Review Letters last month1,2.
Every second, trillions of neutrinos stream through Earth — unnoticed, because they barely interact with ordinary matter. Most of these almost massless particles are produced by fusion reactions in the Sun, such as those that trigger the radioactive β-decay of the isotope boron-8.
Physicists have long predicted that dark-matter experiments will eventually catch a glimpse of this neutrino fog, formerly known as the neutrino floor, says Fei Gao, an experimental particle physicist at Tsinghua University in Beijing. He works on the XENONnt dark-matter experiment at the Gran Sasso National Laboratory just outside L’Aquila, Italy.
The neutrino fog is also exciting because measuring it confirms that dark-matter experiments are capable of observing all ‘flavours’ of neutrinos flying in from the Sun and even from exploding stars in nearby galaxies, says Kate Scholberg, an experimental particle physicist at Duke University in Durham, North Carolina. “You could learn something about the total spectrum of all neutrinos,” she says.
'''
print_summary(text)